In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [3]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [7]:
data_train.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
220,221,1,3,"Sunderland, Mr. Victor Francis",male,16.0,0,0,SOTON/OQ 392089,8.0500,NaN,S
110,111,0,1,"Porter, Mr. Walter Chamberlain",male,47.0,0,0,110465,52.0000,C110,S
662,663,0,1,"Colley, Mr. Edward Pomeroy",male,47.0,0,0,5727,25.5875,E58,S
581,582,1,1,"Thayer, Mrs. John Borland (Marian Longstreth M...",female,39.0,1,1,17421,110.8833,C68,C
675,676,0,3,"Edvardsson, Mr. Gustaf Hjalmar",male,18.0,0,0,349912,7.7750,NaN,S
870,871,0,3,"Balkic, Mr. Cerin",male,26.0,0,0,349248,7.8958,NaN,S
882,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,NaN,S
325,326,1,1,"Young, Miss. Marie Grice",female,36.0,0,0,PC 17760,135.6333,C32,C
128,129,1,3,"Peter, Miss. Anna",female,NaN,1,1,2668,22.3583,F E69,C
876,877,0,3,"Gustafsson, Mr. Alfred Ossian",male,20.0,0,0,7534,9.8458,NaN,S


In [ ]:
Обробити дані для подальшого використання у кластеризації (вирішити проблему з missing data, закодувати змінні).

In [9]:
missing_values = data_train.isnull().sum()
print(missing_values) 

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [20]:
data_train[data_train.duplicated()] 
# дублікатів немає

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [21]:
# будемо заміняти відсутні значення в стовпці "Age", спираючись на кількість дітей у цього пасажира (поле "Parch")
# так як наявність дітей, скоріш за все, свідчить про більший вік

# створимо новий стовпець, що показує, чи є у пасажира діти
data_train['Has_children'] = np.where(data_train['Parch'] > 0, 1, 0)

# розрахуємо середній вік для кожної групи з дітьми та без дітей
mean_age_children = data_train[data_train['Has_children'] == 1]['Age'].mean()
mean_age_no_children = data_train[data_train['Has_children'] == 0]['Age'].mean()

# функція для заміни відсутніх значень віку залежно від наявності дітей
def impute_age(row):
    if pd.isnull(row['Age']):
        if row['Has_children'] == 1:
            return mean_age_children
        else:
            return mean_age_no_children
    else:
        return row['Age']

# замінемо відсутні значень віку за допомогою нової функції
data_train['Age'] = data_train.apply(impute_age, axis=1)

# видяляємо стовпець
data_train.drop('Has_children', axis=1, inplace=True)

In [30]:
# стовпець "Embarked" вказує на порт посадки пасажира (C = Cherbourg, Q = Queenstown, S = Southampton) 
# оскільки в ньому лише два відсутніх значення, можна замінити їх на найбільш поширений порт посадки серед усіх пасажирів. 
most_common_port = data_train['Embarked'].mode()[0]
data_train['Embarked'] = data_train['Embarked'].fillna(most_common_port)

In [31]:
# стовпець "Cabin" містить багато відсутніх значень (687 із 891), тому його доцільніше просто видалити
data_train.drop('Cabin', axis=1, inplace=True)

In [32]:
missing_values = data_train.isnull().sum()
print(missing_values) 

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


In [33]:
data_train.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
167,168,0,3,"Skoog, Mrs. William (Anna Bernhardina Karlsson)",female,45.0,1,4,347088,27.9000,S
881,882,0,3,"Markun, Mr. Johann",male,33.0,0,0,349257,7.8958,S
543,544,1,2,"Beane, Mr. Edward",male,32.0,1,0,2908,26.0000,S
720,721,1,2,"Harper, Miss. Annie Jessie ""Nina""",female,6.0,0,1,248727,33.0000,S
99,100,0,2,"Kantor, Mr. Sinai",male,34.0,1,0,244367,26.0000,S


In [35]:
from sklearn.preprocessing import LabelEncoder

In [36]:
# закодуємо категоріальні змінні "Sex" і "Embarked" у числові формати, щоб наш алгоритм кластеризації міг працювати з цими даними
label_encoder_sex = LabelEncoder()
label_encoder_embarked = LabelEncoder()

data_train['Sex'] = label_encoder_sex.fit_transform(data_train['Sex'])
data_train['Embarked'] = label_encoder_embarked.fit_transform(data_train['Embarked'])

In [37]:
data_train.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
613,614,0,3,"Horgan, Mr. John",1,32.178503,0,0,370377,7.7500,1
496,497,1,1,"Eustis, Miss. Elizabeth Mussey",0,54.000000,1,0,36947,78.2667,0
308,309,0,2,"Abelson, Mr. Samuel",1,30.000000,1,0,P/PP 3381,24.0000,0
174,175,0,1,"Smith, Mr. James Clinch",1,56.000000,0,0,17764,30.6958,0
222,223,0,3,"Green, Mr. George Henry",1,51.000000,0,0,21440,8.0500,2
